In [1]:
import pandas as pd

In [11]:
import json

In [3]:
pd.__version__

'1.1.4'

In [4]:
df = pd.read_csv('iacr.csv')

In [5]:
df.head()

,web-scraper-order,web-scraper-start-url,article,article-href,title,authors,body
0,1604473310-14182,https://eprint.iacr.org/2005,2005/158,https://eprint.iacr.org/2005/158,Geometric Cryptosystem,Arkady Berenstein and Leon Chernyak,"[{""body"":""Abstract: In this paper we propo..."
1,1604475645-15233,https://eprint.iacr.org/2002,2002/100,https://eprint.iacr.org/2002/100,Encryption-Scheme Security in the Presence...,J. Black and P. Rogaway and T. Shrimpton,"[{""body"":""Abstract: Encryption that is only..."
2,1604456951-6776,https://eprint.iacr.org/2015,2015/175,https://eprint.iacr.org/2015/175,Trivial Nonce-Misusing Attack on Pure OMD,Tomer Ashur and Bart Mennink,"[{""body"":""Abstract: Pure OMD is an authent..."
3,1604468473-11994,https://eprint.iacr.org/2009,2009/120,https://eprint.iacr.org/2009/120,On the Complexity of Khovratovich et.al's...,Danilo Gligoroski and Rune Steinsmo Ø de...,"[{""body"":""Abstract: Based on the analysis ..."
4,1604471084-13176,https://eprint.iacr.org/2007,2007/119,https://eprint.iacr.org/2007/119,Rerandomizable RCCA Encryption,Manoj Prabhakaran and Mike Rosulek,"[{""body"":""Abstract: We give the first perf..."


In [8]:
articles = df.drop(['web-scraper-order', 'web-scraper-start-url', 'article'], axis=1)

In [10]:
articles['body'][0]

'[{"body":"Abstract: In  this  paper  we  propose  a  new  class  of  cryptosystems  that  utilizes  metric  continuity.  The  geometric  cryptosystem  considered  in  this  paper  as  the  main  example  of  metric  cryptosystems  has  a  number  of  interesting  properties  such  as  resistance  to  several  basic  cryptographic  attacks,  efficiency   and  detection  of  transmission  errors."},{"body":"Category / Keywords: foundations /"},{"body":"Date: received 16 May 2005"},{"body":"Contact author: arkadiy at math uoregon edu"},{"body":"Available format(s): PDF  | BibTeX Citation"},{"body":"Note: 28 pages"},{"body":"Version: 20050529:213209 (All versions of this report)"},{"body":"Short URL: ia.cr/2005/158"},{"body":""}]'

In [40]:
def get_keywords(body):
    prefix = "Category / Keywords: "
    for item in body:
        if item["body"].startswith(prefix):
            return item["body"][len(prefix):].strip()
    return "Not found"

In [21]:
body = json.loads(articles['body'][0])
get_keyword(body)

'foundations /'

In [26]:
def get_extension(body):
    prefix = "Available format(s):"
    for item in body:
        if item["body"].startswith(prefix):
            content = item["body"][len(prefix):].lower()
            if "pdf" in content:
                return "pdf"
            if "ps" in content:
                return "ps"
    return "invalid" 

In [27]:
get_extension(body)

'pdf'

In [28]:
def get_abstract(body):
    prefix = "Abstract: "
    abstract = ""
    for item in body:
        if item["body"].startswith("Category / Keywords: "):
            break
        abstract += item["body"]
    return abstract[len(prefix):].strip()

In [29]:
get_abstract(body)

'In  this  paper  we  propose  a  new  class  of  cryptosystems  that  utilizes  metric  continuity.  The  geometric  cryptosystem  considered  in  this  paper  as  the  main  example  of  metric  cryptosystems  has  a  number  of  interesting  properties  such  as  resistance  to  several  basic  cryptographic  attacks,  efficiency   and  detection  of  transmission  errors.'

In [32]:
articles.head()

,article-href,title,authors,body
0,https://eprint.iacr.org/2005/158,Geometric Cryptosystem,Arkady Berenstein and Leon Chernyak,"[{""body"":""Abstract: In this paper we propo..."
1,https://eprint.iacr.org/2002/100,Encryption-Scheme Security in the Presence...,J. Black and P. Rogaway and T. Shrimpton,"[{""body"":""Abstract: Encryption that is only..."
2,https://eprint.iacr.org/2015/175,Trivial Nonce-Misusing Attack on Pure OMD,Tomer Ashur and Bart Mennink,"[{""body"":""Abstract: Pure OMD is an authent..."
3,https://eprint.iacr.org/2009/120,On the Complexity of Khovratovich et.al's...,Danilo Gligoroski and Rune Steinsmo Ø de...,"[{""body"":""Abstract: Based on the analysis ..."
4,https://eprint.iacr.org/2007/119,Rerandomizable RCCA Encryption,Manoj Prabhakaran and Mike Rosulek,"[{""body"":""Abstract: We give the first perf..."


In [36]:
articles['body'] = articles['body'].apply(json.loads)

In [41]:
articles['extension'] = articles['body'].apply(get_extension)
articles['abstract'] = articles['body'].apply(get_abstract)
articles['keywords'] = articles['body'].apply(get_keywords)

In [43]:
articles['url'] = articles['article-href'] + "." + articles['extension']

In [46]:
articles = articles.drop(['article-href', 'body', 'extension'], axis=1)

In [75]:
def cleanup_title(title):
    try:
        title = str(title)
        title = title.strip()
        title = ' '.join(title.split())
        return title
    except Exception as e:
        print(e)

In [77]:
articles['title'] = articles['title'].apply(cleanup_title)

In [78]:
articles['authors'] = articles['authors'].apply(str)

In [80]:
articles.to_csv('processed_iacr.csv')

In [81]:
new_df = pd.read_csv('processed_iacr.csv')

In [82]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15566 entries, 0 to 15565
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  15566 non-null  int64 
 1   title       15560 non-null  object
 2   authors     15561 non-null  object
 3   abstract    15563 non-null  object
 4   keywords    15566 non-null  object
 5   url         15566 non-null  object
dtypes: int64(1), object(5)
memory usage: 729.8+ KB
